Cel Projektu:

Zrozumieć, na co użytkownicy wydają pieniądze, które kategorie generują największe koszty
oraz gdzie istnieje potencjał oszczędności.

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

N = 1200

df = pd.DataFrame({
	"transaction_id": range(1, N+1),
	"user_id": np.random.randint(1, 101, size = N),
	"category": np.random.choice(
		["food", "rent", "transport", "entertainment", "health", "subscriptions"],
		size = N,
		p = [0.3, 0.25, 0.15, 0.15, 0.1, 0.05]
	),
	"amount": np.round(np.random.exponential(scale=80, size = N), 2),
	"payment_method": np.random.choice(
		["card", "cash", "transfer"],
		size = N,
		p=[0.6, 0.25, 0.15]
	),
	"is_recurring": np.random.choice([0, 1], size = N, p=[0.7, 0.3]),
	"month": np.random.choice(
		["Jan", "Feb", "Mar", "Apr", "May", "Jun"],
		size=N
	)
})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  1200 non-null   int64  
 1   user_id         1200 non-null   int32  
 2   category        1200 non-null   object 
 3   amount          1200 non-null   float64
 4   payment_method  1200 non-null   object 
 5   is_recurring    1200 non-null   int64  
 6   month           1200 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(3)
memory usage: 61.1+ KB


ETAP 1 EDA

	1. Podstawowy opis danych


In [ ]:
df.head()

,transaction_id,user_id,category,amount,payment_method,is_recurring,month
0,1,52,rent,4.62,transfer,1,Mar
1,2,93,rent,153.24,cash,0,Apr
2,3,15,entertainment,20.45,cash,0,Jan
3,4,72,food,37.85,cash,0,Jan
4,5,61,transport,49.55,card,0,Mar


In [ ]:
df.shape

# Nasza tabela danych ma 1200 wierszy i 7 kolumn

(1200, 7)

In [ ]:
df.info()

# Nasze dane są pełne, nie ma braków


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  1200 non-null   int64  
 1   user_id         1200 non-null   int32  
 2   category        1200 non-null   object 
 3   amount          1200 non-null   float64
 4   payment_method  1200 non-null   object 
 5   is_recurring    1200 non-null   int64  
 6   month           1200 non-null   object 
dtypes: float64(1), int32(1), int64(2), object(3)
memory usage: 61.1+ KB


Statystyki kwot (amount)

In [ ]:
df["amount"].describe()

# Średnia wynosi 81.46
# Mediana wynosi 57.92
# Min wynosi 0.02
# Max wynosi 652.71
# kwartyle: 25% to 23.18, 50% to 57.92, 75% to 114.03


count    1200.000000
mean       81.459817
std        82.185594
min         0.020000
25%        23.185000
50%        57.920000
75%       114.032500
max       652.710000
Name: amount, dtype: float64

 3. Rozkład kategorii wydatków

In [14]:
# Liczba transakcji per category

df["category"].value_counts()

category
food             351
rent             299
transport        197
entertainment    187
health           103
subscriptions     63
Name: count, dtype: int64

In [ ]:
# Łączna kwota per category

df.groupby("category")["amount"].sum().sort_values(ascending=False)

# Najwięcej pieniędzy wydajemy na food (jedzenie)
# Najczęściej wydajemy pieniądze na food i na rent


category
food             28487.49
rent             24718.99
transport        15602.96
entertainment    14049.37
health            8845.38
subscriptions     6047.59
Name: amount, dtype: float64

4. Metody płatności

In [21]:
# rozkład payment_method

df["payment_method"].value_counts()

payment_method
card        718
cash        306
transfer    176
Name: count, dtype: int64

In [ ]:
# Średnia kwota transakcji per metoda

df.groupby("payment_method")["amount"].mean().sort_values(ascending=False)

# Wniosek: Najmniej płacą gotówką a najwięcej przelewem


payment_method
transfer    83.193011
card        83.132618
cash        76.537876
Name: amount, dtype: float64

5. Wydatki w czasie (miesiące)


In [36]:
# Liczba transakcji per month

df.groupby("month")["transaction_id"].count().sort_values(ascending=False)


month
Feb    234
Jan    198
May    196
Mar    194
Jun    190
Apr    188
Name: transaction_id, dtype: int64

In [30]:
# Suma wydatków per month

df.groupby("month")["amount"].sum().sort_values(ascending=False)


month
Feb    18370.86
May    18213.64
Jan    16087.43
Apr    15480.04
Mar    14879.11
Jun    14720.70
Name: amount, dtype: float64

6. Recurring vs non-recurring

In [33]:
# Ile % transakcji jest is_recurring = 1

is_recurring = (df["is_recurring"] == 1).mean() * 100
print(f"{is_recurring:.2f} % transakcji jest is_recurring = 1")

29.42 % transakcji jest is_recurring = 1


In [ ]:
# Średnia kwota: recurring vs non_recurring

df.groupby("is_recurring")["amount"].mean()

# Porównanie średniej transakcji:
# - recurring (np. subskrypcje, rachunki)
# - non_recurring (wydatki jednorazowe)
# Pozwala ocenić, które wydatki bardziej obiążają budżet w długim terminie

is_recurring
0    84.239965
1    74.789037
Name: amount, dtype: float64

ETAP 2 - Agregacje per użytkownik & Struktura wydaktów

1. Wydatki per użytkownik

In [50]:
# liczba transakcji per uzytkownik

transactions_per_user = df.groupby("user_id").agg(
	liczba_transakcji = ("transaction_id", "count")
)

print(transactions_per_user)


         liczba_transakcji
user_id                   
1                       18
2                       15
3                       14
4                       14
5                       16
...                    ...
96                      13
97                      15
98                       8
99                      19
100                     11

[100 rows x 1 columns]


In [55]:
# Suma wydatkow per user

total_spend_per_user = df.groupby("user_id").agg(
	Suma_wydatkow = ("amount", "sum")
)

print(total_spend_per_user)

         Suma_wydatkow
user_id               
1              1577.82
2              1101.76
3              1943.17
4              1209.74
5              1345.78
...                ...
96             1085.65
97             1030.30
98              708.34
99             1500.18
100            1208.52

[100 rows x 1 columns]


In [58]:
# Średnia kwota transakcji per user

avg_transaction_per_user = df.groupby("user_id").agg(
	Srednia_kwota_transakcji = ("amount", "mean")
)

print(avg_transaction_per_user)

         Srednia_kwota_transakcji
user_id                          
1                       87.656667
2                       73.450667
3                      138.797857
4                       86.410000
5                       84.111250
...                           ...
96                      83.511538
97                      68.686667
98                      88.542500
99                      78.956842
100                    109.865455

[100 rows x 1 columns]


In [60]:
# Maksymalny wydatek per user

max_spend_per_user = df.groupby("user_id").agg(
	Maksymalny_wydatek_per_user = ("amount", "max")
)

print(max_spend_per_user)

         Maksymalny_wydatek_per_user
user_id                             
1                             308.90
2                             236.69
3                             330.02
4                             290.32
5                             356.65
...                              ...
96                            282.53
97                            193.68
98                            246.39
99                            282.59
100                           198.91

[100 rows x 1 columns]
